In [1]:
import pickle

import mlflow
import pandas as pd
from sklearn import metrics

In [2]:
STAGE = "Production"
MODEL_NAME = "churn_model"
MODEL_REGISRTY_PATH = f"models:/{MODEL_NAME}/{STAGE}"
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
MLFLOW_EXPERIMENT_NAME = "telco-customers-churn"

In [3]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [72]:
with open("models/min_max_scaler.bin", "rb") as f:
    scaler = pickle.load(f)

with open("models/ohe.bin", "rb") as f:
    ohe = pickle.load(f)


def prepare_features(users: dict):
    num_features = ["tenure", "MonthlyCharges", "TotalCharges"]

    cat_features = [
        "gender",
        "SeniorCitizen",
        "Partner",
        "Dependents",
        "PhoneService",
        "MultipleLines",
        "InternetService",
        "OnlineSecurity",
        "OnlineBackup",
        "DeviceProtection",
        "TechSupport",
        "StreamingTV",
        "StreamingMovies",
        "Contract",
        "PaperlessBilling",
        "PaymentMethod",
    ]
    df = pd.DataFrame.from_dict(users, orient="index").reset_index(drop=True)
    df.TotalCharges = pd.to_numeric(df.TotalCharges, errors="coerce")
    df = df.iloc[:, 1:]
    X = ohe.transform(df[cat_features])
    X = pd.DataFrame(X, columns=ohe.get_feature_names_out())
    X = pd.concat([X, df[num_features]], axis=1)
    features = X.columns.values
    X = pd.DataFrame(scaler.transform(X))
    X.columns = features
    return X

In [40]:
production_model = mlflow.pyfunc.load_model(MODEL_REGISRTY_PATH)

In [41]:
df = pd.read_csv("data/test.csv")
df = df.dropna()
df["Churn"] = df["Churn"].replace(to_replace="Yes", value=1)
df["Churn"] = df["Churn"].replace(to_replace="No", value=0)

In [42]:
X = prepare_features(df.to_dict(orient="index"))

In [43]:
preds = production_model.predict(X)

In [ ]:
metrics.f1_score(preds, df.Churn.values)

0.582312925170068